In [7]:
pip install statsmodels

     ---------------------------------------- 9.1/9.1 MB 17.2 MB/s eta 0:00:00
     ------------------------------------- 233.7/233.7 KB 14.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import metrics
from sklearn.metrics import r2_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

import re
import os

In [23]:
movies = pd.read_csv(r'C:/Users/Timothee TOUMANI/Desktop/work/work_projects/poc_dh/movie_reco/ml-latest-small/movies.csv')
ratings = pd.read_csv(r'C:/Users/Timothee TOUMANI/Desktop/work/work_projects/poc_dh/movie_reco/ml-latest-small/ratings.csv')
tags = pd.read_csv(r'C:/Users/Timothee TOUMANI/Desktop/work/work_projects/poc_dh/movie_reco/ml-latest-small/tags.csv')

### Content Base

In [16]:
# récupérer l'année dans le titre
def getYear(title):
    result = re.search(r'\(\d{4}\)', title)
    if result:
        found = result.group(0).strip('(').strip(')')
    else: 
        found = 0
    return int(found)
    
# retirer l'année du titre
def removeyear(string):
    result=re.search(r'(\d{4})',string)
    if result:
        return string[:-6].strip()
    return string

def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

def get_index_from_title(title):
    return df[df.title == title].index.values[0]


def get_important_features(data):
  important_features = []
  for i in range(0, data.shape[0]):
    important_features.append(data['title'][i]+' '+data['genres'][i])

  return important_features

In [25]:
movies['year'] = movies.apply(lambda x: getYear(x['title']), axis=1)
movies['title'] = movies.title.apply(removeyear)

features = ['title','genres']

movies['features'] = get_important_features(movies)

In [19]:
cm = CountVectorizer().fit_transform(movies['features'])
cs = cosine_similarity(cm)

In [20]:
np.savetxt('cosine_sim.csv', cs, delimiter=',')

### Collaborative filtering

In [26]:
ratings = pd.merge(movies,ratings).drop(['genres','timestamp', 'year', 'features'],axis=1)
print(ratings.shape)
ratings.head()

(100836, 4)


,movieId,title,userId,rating
0,1,Toy Story,1,4.0
1,1,Toy Story,5,4.0
2,1,Toy Story,7,4.5
3,1,Toy Story,15,2.5
4,1,Toy Story,17,4.5


In [27]:
userRatings = ratings.pivot_table(index=['userId'],columns=['movieId'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(userRatings.mean())
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (610, 9724)
After:  (610, 2269)


In [28]:
corrMatrix = userRatings.corr(method='pearson')

In [29]:
corrMatrix.to_csv('matrice2.csv',index = True, index_label ='movieId')

In [30]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings